In [1]:
import pandas as pd
import sqlite3
import os

subject = 'IorNYll1lS' #! The ID of the subject on Back4app
folder_path = 'database extractor/data'
df_file = '20240327_091233_14ChannelSensorDatabase_mci014.db'
db_file_path = f'{folder_path}/{df_file}'


In [2]:
# Create a connection to the SQLite database
conn = sqlite3.connect(db_file_path)

# Write your SQL query (replace 'your_table_name' with the actual table name)
sql_query = 'SELECT * FROM USER_SESSION_TABLE WHERE SESSION_ID BETWEEN 206 AND 364'

# Use pandas to read data from the database into a DataFrame
df = pd.read_sql(sql_query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
print(df)

     SESSION_ID                     USERNAME DEVICE_ADDRESS_LEFT  \
0           206  flexosensephone18@gmail.com   FF:1C:BB:9A:EE:CB   
1           207  flexosensephone18@gmail.com   FF:1C:BB:9A:EE:CB   
2           208  flexosensephone18@gmail.com   FF:1C:BB:9A:EE:CB   
3           209  flexosensephone18@gmail.com   FF:1C:BB:9A:EE:CB   
4           210  flexosensephone18@gmail.com   FF:1C:BB:9A:EE:CB   
..          ...                          ...                 ...   
154         360      flexosense18p@gmail.com   FF:1C:BB:9A:EE:CB   
155         361      flexosense18p@gmail.com   FF:1C:BB:9A:EE:CB   
156         362      flexosense18p@gmail.com   FF:1C:BB:9A:EE:CB   
157         363      flexosense18p@gmail.com   FF:1C:BB:9A:EE:CB   
158         364      flexosense18p@gmail.com   FF:1C:BB:9A:EE:CB   

    DEVICE_ADDRESS_RIGHT START_DATE START_TIME START_BATTERY  END_DATE  \
0      C2:B6:2D:8F:53:1F   20240229     112657       100,100  20240229   
1      C2:B6:2D:8F:53:1F   20240229

In [3]:
# Create a connection to the SQLite database
conn = sqlite3.connect(db_file_path)

# Write SQL queries for each table
right_insole_query = 'SELECT * FROM RIGHT_INSOLE_RAW WHERE SESSION_ID BETWEEN 206 AND 364'
left_insole_query = 'SELECT * FROM LEFT_INSOLE_RAW WHERE SESSION_ID BETWEEN 206 AND 364'

# Use pandas to read data from each table into DataFrames
right_insole_df = pd.read_sql(right_insole_query, conn)
left_insole_df = pd.read_sql(left_insole_query, conn)

# Close the database connection
conn.close()

# Group the DataFrames by SESSION_ID
grouped_right_insole = right_insole_df.groupby('SESSION_ID').agg(list).reset_index()
grouped_left_insole = left_insole_df.groupby('SESSION_ID').agg(list).reset_index()

# Display the grouped DataFrames
print("Grouped RIGHT_INSOLE_RAW:")
print(grouped_right_insole)

print("\nGrouped LEFT_INSOLE_RAW:")
print(grouped_left_insole)

Grouped RIGHT_INSOLE_RAW:
     SESSION_ID                                             RAW_ID  \
0           206  [8216320, 8216321, 8216322, 8216323, 8216324, ...   
1           207  [8217262, 8217263, 8217264, 8217265, 8217266, ...   
2           208  [8226124, 8226125, 8226126, 8226127, 8226128, ...   
3           209  [8227642, 8227643, 8227644, 8227645, 8227646, ...   
4           210  [8298967, 8298968, 8298969, 8298970, 8298971, ...   
..          ...                                                ...   
154         360  [17260379, 17260380, 17260381, 17260382, 17260...   
155         361  [17260794, 17260795, 17260796, 17260797, 17260...   
156         362  [17332589, 17332590, 17332591, 17332592, 17332...   
157         363  [17404589, 17404590, 17404591, 17404592, 17404...   
158         364  [17476586, 17476587, 17476588, 17476589, 17476...   

                                             READING_1  \
0    [-4, -3, -3, -1, -3, -1, -4, -1, -3, -3, -2, -...   
1    [-3, -3, -4,

In [4]:
# Create a connection to the SQLite database
conn = sqlite3.connect(db_file_path)

# Read USER_SESSION_TABLE to get STARTDATE for each SESSION_ID
user_session_query = 'SELECT SESSION_ID, START_DATE FROM USER_SESSION_TABLE'
user_session_df = pd.read_sql(user_session_query, conn)

# Merge user_session_df with grouped_right_insole and grouped_left_insole
merged_right_insole = pd.merge(grouped_right_insole, user_session_df, on='SESSION_ID', how='inner')
merged_left_insole = pd.merge(grouped_left_insole, user_session_df, on='SESSION_ID', how='inner')

# Close the database connection
conn.close()

# Specify the range of SESSION_ID values
start_session_id = 206
end_session_id = 364

# Root folder path
root_folder_path = 'database extractor/data/mci006/'

# Filter data for the specified SESSION_ID range
filtered_right_insole = merged_right_insole[(merged_right_insole['SESSION_ID'] >= start_session_id) & (merged_right_insole['SESSION_ID'] <= end_session_id)]
filtered_left_insole = merged_left_insole[(merged_left_insole['SESSION_ID'] >= start_session_id) & (merged_left_insole['SESSION_ID'] <= end_session_id)]

# Iterate through each row and save data as TXT in the specified folder
for index, session in filtered_right_insole.iterrows():
    session_id = session['SESSION_ID']
    start_date = str(session['START_DATE'])
    date_folder = start_date[:4] + '-' + start_date[4:6] + '-' + start_date[6:]
    array_data = session.drop(['SESSION_ID', 'START_DATE', 'RAW_ID'])
    session_df = pd.DataFrame(array_data).T
    folder_path = os.path.join(root_folder_path, date_folder)
    
    # Convert each array to pandas Series
    series_list = {col: pd.Series(data) for col, data in array_data.items()}
    if len(series_list['READING_1']) < 2 / 0.05: #! data can be less than 2 seconds, which is not necessary
        print(f'Session {session_id} less than 2 seconds, Skip')
        print("-----------------------------------------------------------")
        continue
    # Create a new DataFrame by concatenating the Series
    session_df = pd.DataFrame(series_list)
    
    os.makedirs(folder_path, exist_ok=True)  # Create folder if it doesn't exist
    filename = os.path.join(folder_path, f'S{session_id}_{start_date}_rawDataRight_{subject}.txt')
    session_df.to_csv(filename, sep=',', header=False, index=False)

for index, session in filtered_left_insole.iterrows():
    session_id = session['SESSION_ID']
    start_date = str(session['START_DATE'])
    date_folder = start_date[:4] + '-' + start_date[4:6] + '-' + start_date[6:]
    array_data = session.drop(['SESSION_ID', 'START_DATE', 'RAW_ID'])
    session_df = pd.DataFrame(array_data).T
    folder_path = os.path.join(root_folder_path, date_folder)
    
    # Convert each array to pandas Series
    series_list = {col: pd.Series(data) for col, data in array_data.items()}
    if len(series_list['READING_1']) < 2 / 0.05: #! data can be less than 2 seconds, which is not necessary
        print(f'Session {session_id} less than 2 seconds, Skip')
        print("-----------------------------------------------------------")
        continue
    # Create a new DataFrame by concatenating the Series
    session_df = pd.DataFrame(series_list)
    
    os.makedirs(folder_path, exist_ok=True)  # Create folder if it doesn't exist
    filename = os.path.join(folder_path, f'S{session_id}_{start_date}_rawDataLeft_{subject}.txt')
    session_df.to_csv(filename, sep=',', header=False, index=False)